In [57]:
%pip install polars
%pip install geopandas
%pip install matplotlib
%pip install folium matplotlib mapclassify

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
     -------------------------------------- 108.4/108.4 kB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 59.1/59.1 kB 3.3 MB/s eta 0:00:00
     -------------------------------------- 133.3/133.3 kB 4.0 MB/s eta 0:00:00
     ---------------------------------------- 64.9/64.9 kB ? eta 0:00:00
     ---------------------------------------- 85.1/85.1 kB ? eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 15.6 MB/s eta 0:00:00
     --------------------------------------- 11.0/11.0 MB 40.9 MB/s eta 0:00:00
     --------------------------------------- 44.8/44.8 MB 25.2 MB/s eta 0:00:00
     ------------------------------------- 301.8/301.8 kB 18.2 MB/s eta 0:00:00
     ---------------------------------------- 101.8/101.8 kB ? eta 0:00:00
     ---------------------------------------- 70.4/70.4 kB ? eta 0:00:00
     -------------------------------------- 126.3/126.3 kB 7.3 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [80]:
import polars as pl
import geopandas as gpd

pl.Config.set_tbl_rows(200)

polars.config.Config

In [152]:
metric_dfs = {}

In [153]:
metric_dfs["population_statistics"] = (
    pl.read_csv("https://data.stat.gov.lv:443/sq/23396", skip_rows=2)
    .with_columns(
        *[
            pl.col(col).replace("…", None).cast(pl.Int64)
            for col in [
                'Gada vidējais iedzīvotāju skaits',
                'Iedzīvotāju dabiskais pieaugums',
                'Migrācijas saldo',
                'Iedzīvotāju skaita izmaiņas - pavisam'
            ]
        ]
    )
)
metric_dfs["population_statistics"].head()

Teritoriālā vienība,Laika periods,Iedzīvotāju skaits gada sākumā,Gada vidējais iedzīvotāju skaits,Iedzīvotāju dabiskais pieaugums,Migrācijas saldo,Iedzīvotāju skaita izmaiņas - pavisam
str,i64,i64,i64,i64,i64,i64
"""Rīga""",2012,636933,633749,-2182,-4186,-6368
"""Rīga""",2013,630565,631803,-1972,4448,2476
"""Rīga""",2014,633041,632397,-1472,185,-1287
"""Rīga""",2015,631754,631010,-1243,-245,-1488
"""Rīga""",2016,630266,631765,-1159,4157,2998


In [154]:
metric_dfs["work_places"] = (
    pl.read_csv("https://data.stat.gov.lv:443/sq/23400", skip_rows=2)
    .select(
        pl.col("Teritoriālā vienība"),
        pl.col("Laika periods"),
        pl.col("Sabiedriskajā sektorā Pavisam").alias("Aizņemtās darbvietas Sabiedriskajā sektorā"),
        pl.col("Privātajā sektorā Pavisam").alias("Aizņemtās darbvietas Privātajā sektorā")
    )
)
metric_dfs["work_places"].head(5)

Teritoriālā vienība,Laika periods,Aizņemtās darbvietas Sabiedriskajā sektorā,Aizņemtās darbvietas Privātajā sektorā
str,i64,i64,i64
"""Latvija""",2022,284440,603276
"""Latvija""",2023,286922,599531
"""Rīgas statistiskais reģions (R…",2022,148371,331654
"""Rīgas statistiskais reģions (R…",2023,149219,328070
"""Pierīgas statistiskais reģions…",2022,36908,103289


In [155]:
metric_dfs["road_lengths"] = (
    pl.read_csv("https://data.stat.gov.lv:443/sq/23402", skip_rows=2)
    .select(
        pl.col("Teritoriālā vienība"),
        pl.col("Laika periods"),
        pl.col("Pavisam Pavisam").alias("Valsts un pašvaldību autoceļu un pašvaldību ielu garums"),
    )
)
metric_dfs["road_lengths"].head(5)

Teritoriālā vienība,Laika periods,Valsts un pašvaldību autoceļu un pašvaldību ielu garums
str,i64,str
"""Latvija""",2021,"""57461"""
"""Latvija""",2022,"""57971"""
"""Latvija""",2023,"""57906"""
"""Rīgas statistiskais reģions (R…",2021,"""1205"""
"""Rīgas statistiskais reģions (R…",2022,"""1205"""


In [156]:
metric_dfs["farming_land"] = (
    pl.read_csv("https://data.stat.gov.lv:443/sq/23403", skip_rows=2)
    .with_columns(
        pl.col("Teritoriālā vienība").str.split(" ").list.first().alias("code"),
        pl.col("Teritoriālā vienība").str.split(" ").list.slice(1).list.join(" ").alias("Teritoriālā vienība"),
    )
    .select(
        pl.col("Teritoriālā vienība"),
        pl.col("Laika periods"),
        pl.col("Pavisam").alias("Bioloģiskās lauksaimniecības operatoru sertificētās platības"),
    )
)
metric_dfs["farming_land"].head(5)

Teritoriālā vienība,Laika periods,Bioloģiskās lauksaimniecības operatoru sertificētās platības
str,i64,i64
"""Latvija""",2019,289601
"""Latvija""",2020,291469
"""Latvija""",2021,302177
"""Rīgas statistiskais reģions (n…",2019,5521
"""Rīgas statistiskais reģions (n…",2020,6060


In [157]:
metric_dfs["clean_energy"] = (
    pl.read_csv("https://data.stat.gov.lv:443/sq/23404", skip_rows=2)
    .with_columns(
        pl.col("Teritoriālā vienība").str.split(" ").list.first().alias("code"),
        pl.col("Teritoriālā vienība").str.split(" ").list.slice(1).list.join(" ").alias("Teritoriālā vienība"),
    )
    .select(
        pl.col("Teritoriālā vienība"),
        pl.col("Laika periods"),
        pl.col("Pavisam").alias("Saražotā elektroenerģija no atjaunīgiem energoresursiem"),
    )
)
metric_dfs["clean_energy"].head(5)

Teritoriālā vienība,Laika periods,Saražotā elektroenerģija no atjaunīgiem energoresursiem
str,i64,f64
"""Latvija""",2019,3191.4
"""Latvija""",2020,3648.66
"""Latvija""",2021,3716.666
"""Latvija""",2022,3764.778
"""Latvija""",2023,4846.611


In [158]:
metric_dfs["real_estate_transactions"] = (
    pl.read_csv("https://data.stat.gov.lv:443/sq/23405", skip_rows=2, null_values=["…"])
    .select(
        pl.col("Teritoriālā vienība"),
        pl.col("Laika periods"),
        pl.exclude(["Teritoriālā vienība", "Laika periods"]).name.prefix("Nekustamā īpašuma pirkuma līgumi: ")
    )
)
metric_dfs["real_estate_transactions"].head(5)

Teritoriālā vienība,Laika periods,Nekustamā īpašuma pirkuma līgumi: Skaits Apbūves tiesība,Nekustamā īpašuma pirkuma līgumi: Skaits Dzīvokļa īpašums,"Nekustamā īpašuma pirkuma līgumi: Skaits Neapdzīvojamās telpas (garāža, veikals, darbnīca)",Nekustamā īpašuma pirkuma līgumi: Skaits Patstāvīgs ēkas nekustams īpašums,Nekustamā īpašuma pirkuma līgumi: Skaits Privatizējamā objekta sākotnējā reģistrācija,Nekustamā īpašuma pirkuma līgumi: Skaits Zemes nekustams īpašums,Nekustamā īpašuma pirkuma līgumi: Skaits Zemes un ēkas nekustams īpašums,Nekustamā īpašuma pirkuma līgumi: Kopējā summa (eiro) Apbūves tiesība,Nekustamā īpašuma pirkuma līgumi: Kopējā summa (eiro) Dzīvokļa īpašums,"Nekustamā īpašuma pirkuma līgumi: Kopējā summa (eiro) Neapdzīvojamās telpas (garāža, veikals, darbnīca)",Nekustamā īpašuma pirkuma līgumi: Kopējā summa (eiro) Patstāvīgs ēkas nekustams īpašums,Nekustamā īpašuma pirkuma līgumi: Kopējā summa (eiro) Privatizējamā objekta sākotnējā reģistrācija,Nekustamā īpašuma pirkuma līgumi: Kopējā summa (eiro) Zemes nekustams īpašums,Nekustamā īpašuma pirkuma līgumi: Kopējā summa (eiro) Zemes un ēkas nekustams īpašums
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""Latvija""",2021,12,22589,2324,2197,1,21747,11788,939121,997928165,37510881,74673809,5000,579161333,832048748
"""Latvija""",2022,23,21969,2478,2093,3,19370,10591,2616156,1069985340,36890874,80867552,38013,631083054,845489335
"""Latvija""",2023,26,20449,2107,1857,4,16561,8496,1493192,1005166843,38360968,69001158,86700,565840823,656289740
"""Rīga""",2021,1,10379,1715,437,null,3532,1695,446700,679240108,28753742,36907017,null,165211171,309928128
"""Rīga""",2022,1,10144,1899,451,null,3088,1721,67500,719504888,29539447,40211523,null,178322440,313393118


In [159]:
metric_dfs["forests"] = (
    pl.read_csv("https://data.stat.gov.lv:443/sq/23406", skip_rows = 2)
)
metric_dfs["forests"].head(5)

Teritoriālā vienība,Laika periods,Mežs Pavisam
str,i64,i64
"""Latvija""",2021,3094212
"""Latvija""",2022,3108954
"""Latvija""",2023,3121268
"""Rīga""",2021,5460
"""Rīga""",2022,5455


In [160]:
metric_dfs["tourism"] = (
    pl.read_csv("https://data.stat.gov.lv:443/sq/23407", skip_rows=2)
    .select(
        pl.col("Teritoriālā vienība"),
        pl.col("Laika periods"),
        pl.exclude(["Teritoriālā vienība", "Laika periods"]).name.prefix("Viesnīcas un citas tūristu mītnes: ")
    )
)
metric_dfs["tourism"].head(5)

Teritoriālā vienība,Laika periods,Viesnīcas un citas tūristu mītnes: Mītņu skaits (perioda beigās),Viesnīcas un citas tūristu mītnes: Gultasvietu skaits (perioda beigās),Viesnīcas un citas tūristu mītnes: Apkalpotās personas
str,i64,i64,i64,str
"""Latvija""",2022,786,38481,"""2172069"""
"""Latvija""",2023,851,41364,"""2435795"""
"""Rīgas statistiskais reģions (R…",2022,93,13359,"""1050121"""
"""Rīgas statistiskais reģions (R…",2023,104,14632,"""1244176"""
"""Pierīgas statistiskais reģions…",2022,186,9120,"""466306"""


In [164]:
metric_df = list(metric_dfs.values())[0]

for other_df in list(metric_dfs.values())[1:]:
    metric_df = metric_df.join(other_df, on=["Teritoriālā vienība", "Laika periods"], how="full", coalesce=True)

In [168]:
metric_df.write_csv("data/metrics.csv")

In [89]:
geo = gpd.read_file("data/novadi_all_metrics.zip")
# geo.columns = ["code", "Novads", "pop", "geometry"]
# del geo["code"]
# del geo["pop"]
